<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Fallstudie_Juicy_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG

In [1]:
! pip install -q pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 76.6 MB/s eta 0:00:00


In [2]:
from pyscipopt import Model, quicksum

## Indexmengen

## Parameter

In [5]:
kosten = {}
kosten[("Charleston", "USA")] = 81
kosten[("Charleston", "Südamerika")] = 92
kosten[("Charleston", "Europa")] = 101
kosten[("Charleston", "Asien")] = 130
kosten[("Charleston", "Afrika")] = 115

kosten[("Curitiba", "USA")] = 117
kosten[("Curitiba", "Südamerika")] = 77
kosten[("Curitiba", "Europa")] = 108
kosten[("Curitiba", "Asien")] = 98
kosten[("Curitiba", "Afrika")] = 100

kosten[("Hamburg", "USA")] = 102
kosten[("Hamburg", "Südamerika")] = 105
kosten[("Hamburg", "Europa")] = 95
kosten[("Hamburg", "Asien")] = 119
kosten[("Hamburg", "Afrika")] = 111

kosten[("Quanzhou", "USA")] = 115
kosten[("Quanzhou", "Südamerika")] = 125
kosten[("Quanzhou", "Europa")] = 90
kosten[("Quanzhou", "Asien")] = 59
kosten[("Quanzhou", "Afrika")] = 74

kosten[("East London", "USA")] = 142
kosten[("East London", "Südamerika")] = 100
kosten[("East London", "Europa")] = 103
kosten[("East London", "Asien")] = 105
kosten[("East London", "Afrika")] = 71

## Entscheidungsvariablen

## Zielfunktion

## Nebenbedingungen

## Lösung